In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

ePix HR Test Structure Data Analysis

Mode TS 0, single ended
Full Scale

clk_i is 250MHz

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
epixhr test structure data

:Author: Faisal Abu-Nimeh (abunimeh@slac.stanford.edu)
:License: https://confluence.slac.stanford.edu/display/ppareg/LICENSE.html
:Date: 20180629
:Style: OpenStack Style Guidelines https://docs.openstack.org/developer/hacking/
:vcs_id: $Id$
"""

# %matplotlib widget


# import h5py
import numpy as np
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
import os
import logging

# logging.basicConfig(format='%(asctime)s %(levelname)s %(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

np.set_printoptions(formatter={'int': hex})
fig_id=0

Constants

In [ ]:
#------ uint 16 ------
# Packet Size [31:16] 
# Packet Size [15:0]  
# Flags [31:16]       
# Flags [15:0]        
# x00 & Lane Number & VC Number
# x0000 
# Acquisition number [15:0]
# Acquisition number [31:16]
# x000 & 0 & ASIC number
# x0000
# Sample 0
# Sample 1
# ...
# Sample n (i.e. NSAMPLES)
#------ uint 16 ------

NSAMPLES = 30  # Number of ADC Samples in a packet
HEADER_SZ = 10  # number of 16-bit words in header
PKT_SZ = (HEADER_SZ + NSAMPLES) * 2  # Size of epixhr ts packet in bytes
PKT_WRD = NSAMPLES + HEADER_SZ  # number of words in a single packet

dac_resol = 2**16-1
dac_mem = 1024
fscale = 2.5 # 0 to 2.5 volt
tiles = 5

In [ ]:
def loadhrdata(fname):
    pBIN = np.fromfile(fname, dtype=np.uint16)
    # pBIN = np.fromfile(fname)
    filename = os.path.basename(fname)
    mdate = os.path.getmtime(fname)
    logging.debug("uint16 file size: %d" % (pBIN.size))
    
    if pBIN[0] != PKT_SZ:
        logging.error("Invalid frame size, should be: 0x%x." % (PKT_SZ))

    logging.debug("First word: 0x%x." % (pBIN[0]))
    
    step = (PKT_SZ+4)//2  # steps in 16-bit words, add additional byte by streamer
    logging.debug("step is (0x%x) %d 16-bit words" % (step,step))

    pkt_szs = pBIN[::step]
    findx = np.array_equal(pkt_szs, (np.full(pkt_szs.size, PKT_SZ, np.uint16)))
    if not findx:
        logging.error('packet sizes are not equal')
    else:
        logging.debug("Number of packets in file is %d" % (pkt_szs.shape[0]))

    if (PKT_SZ+4) * pkt_szs.shape[0] != pBIN.size*2:
        logging.error('file contains partial frames')

    # # skip headers and keep data only
    pkt_idx = step*np.arange(pkt_szs.shape[0]) + HEADER_SZ
    data_idx = np.arange(NSAMPLES) + pkt_idx[:, np.newaxis]
    data = pBIN[data_idx]
    # drop 1st 0x0 sample
    data = data[:, 1:]
    print(data)
    logging.debug("Data shape is" + str(data.shape))
    return data.copy()

Load data file

In [ ]:
# load dat file
data_ramp_250 = loadhrdata('datadump/20180910_172923.dat')
data_ramp_250_slvds = loadhrdata('datadump/20180910_171620.dat')

In [ ]:
fig_id+=1
sample_id = 20  #  sample of interest
plt.figure(fig_id,figsize=(8,6),dpi=150)
# plt.legend(frameon=False)
plt.xlabel('Samples (Bins)')
plt.ylabel('Amplitude (ADU)')
plt.title('ePix HR Test Structure')
plt.plot(data_ramp_250[:,sample_id])
plt.plot(data_ramp_250_slvds[:,sample_id])


plt.show()

In [ ]:
fig_id+=1
plt.figure(1,figsize=(8,6),dpi=150)
d1_codes = data_ramp_250.ravel()
d2_codes = data_ramp_250_slvds.ravel()

# plt.hist(data_ramp_250.ravel(), label='slvds=false', bins='auto', histtype='step')
# plt.hist(data_ramp_250_slvds.ravel(), label='slvds=true', bins='auto', histtype='step')

plt.hist(d1_codes, label='slvds=false', bins=dac_resol, histtype='step')
plt.hist(d2_codes, label='slvds=true', bins=dac_resol, histtype='step')

plt.xlabel('ADC Output Code')
plt.ylabel('Code Count')
plt.title('ePix HR Histogram, 250MHz, Single-Ended, Ramp Rising')
plt.legend(frameon=False, loc=2)
plt.show()

In [ ]:
d1_codes_vc = np.unique(d1_codes, return_counts=True)
val, cnt = np.unique(d1_codes, return_counts=True)
d2_codes_vc = np.unique(d2_codes, return_counts=True)

print(d1_codes_vc[0])
print(d1_codes_vc[1])

print(val)
print(cnt.max())


d1_bins = np.asarray((d1_codes_vc[0], d1_codes_vc[1])).T
print(d1_bins)
dl_bins_sorted = np.sort(d1_bins, axis=0)
print(dl_bins_sorted)
print(dl_bins_sorted[:,0])
print(dl_bins_sorted[:,1])

In [ ]:
fig_id+=1
plt.figure(fig_id,figsize=(8,6),dpi=150)
plt.xlabel('ADC Output Code')
plt.ylabel('Code Count')
plt.title('ePix HR Histogram, 250MHz, Single-Ended, Ramp Rising')
plt.bar(dl_bins_sorted[:,0], dl_bins_sorted[:,1])
plt.show()


In [ ]:
d1 = np.right_shift(data_ramp_250[:,sample_id],2)
d2 = np.right_shift(data_ramp_250_slvds[:,sample_id],2)

fig_id+=1
sample_id = 20  #  sample of interest
plt.figure(fig_id,figsize=(8,6),dpi=150)
# plt.legend(frameon=False)
plt.xlabel('Samples (Bins)')
plt.ylabel('Amplitude (ADU)')
plt.title('ePix HR Test Structure')
plt.plot(d1)
plt.plot(d2)

plt.show()

In [ ]:
fig_id+=1
plt.figure(1,figsize=(8,6),dpi=150)
plt.hist(d1.ravel(), label='slvds=false', bins=2**14, histtype='step')
plt.hist(d2.ravel(), label='slvds=true', bins=2**14, histtype='step')
plt.xlabel('ADC Output Code')
plt.ylabel('Code Count')
plt.title('ePix HR Histogram, 250MHz, Single-Ended, Ramp Rising, 14-bit MSB')
plt.legend(frameon=False, loc=2)
plt.show()

In [ ]:
val, cnt = np.unique(data_ramp_250.ravel(), return_counts=True)
print(cnt.max())
print(cnt.shape)
print(val.shape)

val, cnt = np.unique(data_ramp_250_slvds.ravel(), return_counts=True)
print(cnt.max())